# Analysis for Total Grade [mean of quizzes and exercises]                    CLUSTER (START)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import scipy.stats as st
import statsmodels.stats.multicomp as ml
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp
import matplotlib
import matplotlib.pyplot as plt 
import scipy.stats as stats
%matplotlib inline

In [5]:
#EΙΣΑΓΩΓΗ ΔΕΔΟΜΕΝΩΝ
df = pd.read_excel('MOOC3.xlsx', sheet_name= "GENERAL", skiprows=1)

df3=df[['Cluster (START)', 'Subgroup', 'QUIZ  1', 'QUIZ  2', 'QUIZ  3', 'QUIZ  4', 'QUIZ  5','Άσκηση  1','Άσκηση 2','Άσκηση 3','Άσκηση 4','Άσκηση5']]
df3
#one= df3.Subgroup==1
#df3_by1= df3[one]
#df3_by1



,Cluster (START),Subgroup,QUIZ 1,QUIZ 2,QUIZ 3,QUIZ 4,QUIZ 5,Άσκηση 1,Άσκηση 2,Άσκηση 3,Άσκηση 4,Άσκηση5
0,2.0,1.0,10,-,-,-,-,10,-,-,-,-
1,2.0,1.0,10,8.67,0,0,0,-,-,-,-,-
2,NaN,NaN,-,-,-,-,-,-,-,-,-,-
3,NaN,NaN,-,-,-,-,-,-,-,-,-,-
4,2.0,1.0,10,10,10,8.75,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...
2655,NaN,NaN,-,-,-,-,-,-,-,-,-,-
2656,2.0,1.0,-,-,-,-,-,-,-,-,-,-
2657,2.0,1.0,10,9.33,7.78,10,10,10,8,10,-,10
2658,NaN,NaN,-,-,-,-,-,-,-,-,-,-


In [6]:
#EΛΕΓΧΟΣ ΓΙΑ ΚΑΘΕ ΜΑΘΗΤΗ ΑΝ ΕΧΕΙ ΑΠΑΝΤΗΣΕΙ ΣΕ ΤΟΥΛΑΧΙΣΤΟΝ 2 ΑΣΚΗΣΕΙΣ ΣΥΝΟΛΙΚΑ. ΑΝ ΕΧΕΙ <2 AΠΑΝΤΗΜΕΝΑ QUIZ ΤΟ ΚΑΝΩ DROP 

exercise_list=['Άσκηση  1','Άσκηση 2','Άσκηση 3','Άσκηση 4','Άσκηση5']
quiz_list=['QUIZ  1', 'QUIZ  2', 'QUIZ  3', 'QUIZ  4', 'QUIZ  5']

for i in range(df3.shape[0]):
    counter_q = 0
    counter_ex = 0
    if df3.at[i,'Subgroup'] == 1:
        
        for j in exercise_list:#ελεγχος για τουλάχιστον δύο quiz
            
            if df3.at[i,j]!="-":             
                counter_ex+=1                       

             
        for j in quiz_list:#ελεγχος για τουλάχιστον δύο ασκήσεις
            
            if df3.at[i,j]!="-":             
                counter_q+=1     
                
        if counter_ex < 2  or counter_q < 2:
            
            df3.drop(df.index[i], inplace=True)    
            
            
    else:
        df3.drop(df.index[i], inplace=True)
        
          
df3

C:\Users\stell\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Cluster (START),Subgroup,QUIZ 1,QUIZ 2,QUIZ 3,QUIZ 4,QUIZ 5,Άσκηση 1,Άσκηση 2,Άσκηση 3,Άσκηση 4,Άσκηση5
4,2.0,1.0,10,10,10,8.75,10,10,10,10,10,10
10,3.0,1.0,8.89,10,10,8.75,6,10,10,10,10,5
15,1.0,1.0,8.89,8,-,-,-,10,2,-,-,-
19,3.0,1.0,10,8,10,8.75,6,8,8,10,3,10
21,1.0,1.0,8.89,8,-,-,-,10,2,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
2642,4.0,1.0,10,9.33,10,10,10,10,8,10,10,10
2647,3.0,1.0,10,9.33,10,10,10,10,6,10,9,8
2648,3.0,1.0,10,9.33,8.06,8.75,10,4,2,10,10,10
2652,3.0,1.0,10,10,10,8.75,10,10,8,10,-,10


In [7]:
df3_grade=df3[['QUIZ  1', 'QUIZ  2', 'QUIZ  3', 'QUIZ  4', 'QUIZ  5','Άσκηση  1','Άσκηση 2','Άσκηση 3','Άσκηση 4','Άσκηση5']]
df3_grade.replace(to_replace='-',value=float('nan'), inplace=True)
df3_grade['mean'] = df3_grade.mean(axis=1, skipna=True)

df3info=df3[['Cluster (START)','Subgroup']]
result= [df3info, df3_grade]
df3_grade_total= pd.concat(result, axis=1)

df3_grade_total

C:\Users\stell\anaconda3\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-7-536bf5379a34>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_grade['mean'] = df3_grade.mean(axis=1, skipna=True)


,Cluster (START),Subgroup,QUIZ 1,QUIZ 2,QUIZ 3,QUIZ 4,QUIZ 5,Άσκηση 1,Άσκηση 2,Άσκηση 3,Άσκηση 4,Άσκηση5,mean
4,2.0,1.0,10.00,10.00,10.00,8.75,10.0,10.0,10.0,10.0,10.0,10.0,9.875000
10,3.0,1.0,8.89,10.00,10.00,8.75,6.0,10.0,10.0,10.0,10.0,5.0,8.864000
15,1.0,1.0,8.89,8.00,NaN,NaN,NaN,10.0,2.0,NaN,NaN,NaN,7.222500
19,3.0,1.0,10.00,8.00,10.00,8.75,6.0,8.0,8.0,10.0,3.0,10.0,8.175000
21,1.0,1.0,8.89,8.00,NaN,NaN,NaN,10.0,2.0,NaN,NaN,NaN,7.222500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2642,4.0,1.0,10.00,9.33,10.00,10.00,10.0,10.0,8.0,10.0,10.0,10.0,9.733000
2647,3.0,1.0,10.00,9.33,10.00,10.00,10.0,10.0,6.0,10.0,9.0,8.0,9.233000
2648,3.0,1.0,10.00,9.33,8.06,8.75,10.0,4.0,2.0,10.0,10.0,10.0,8.214000
2652,3.0,1.0,10.00,10.00,10.00,8.75,10.0,10.0,8.0,10.0,NaN,10.0,9.638889


In [8]:
#Cluster Start 1 (LOW>4)
tg1=df3_grade_total.groupby('Cluster (START)').get_group(1)

tg1
col=tg1.columns
low_grade=tg1[col[12:13:1]]
low_grade.dropna(inplace=True)

meanlow_grade = low_grade.mean(axis=0)#μεσος ορος τελικής βαθμολογίας μαθητων low στις 5 ασκήσεις + 5 Quizzes

low_grade_list=[]
for i, row in low_grade.iterrows():
    low_grade_list.append(low_grade.at[i,'mean'])
low_grade_list

<ipython-input-8-d9b846308ada>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_grade.dropna(inplace=True)


[7.2225,
 7.2225,
 6.453,
 8.511,
 9.4,
 9.05,
 6.769,
 9.024444444444445,
 8.25,
 6.4,
 9.253333333333334,
 5.706,
 7.909999999999999,
 8.543333333333333,
 8.131,
 6.7989999999999995,
 9.267,
 7.617,
 9.534,
 8.741111111111111,
 7.534000000000001,
 7.7219999999999995,
 8.875,
 9.9,
 9.833333333333334,
 7.3,
 8.453,
 9.937999999999999,
 9.564,
 9.356,
 8.195,
 9.733,
 7.586,
 8.097,
 8.495000000000001,
 9.575,
 7.839,
 9.764,
 9.1,
 8.194444444444445,
 8.139,
 8.309000000000001,
 9.925555555555555,
 9.9,
 8.889,
 5.845000000000001,
 8.989,
 7.578]

In [9]:
#Cluster Start 2 (MEDIUM 4-5)
tg2=df3_grade_total.groupby('Cluster (START)').get_group(2)

tg2
col=tg2.columns
medium_grade=tg2[col[12:13:1]]
medium_grade.dropna(inplace=True)

meanmedium_grade = medium_grade.mean(axis=0)#μεσος ορος τελικής βαθμολογίας μαθητων medium στις 5 ασκήσεις + 5 Quizzes

medium_grade_list=[]
for i, row in medium_grade.iterrows():
    medium_grade_list.append(medium_grade.at[i,'mean'])
medium_grade_list

<ipython-input-9-250894f74e54>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medium_grade.dropna(inplace=True)


[9.875,
 9.378,
 7.861,
 9.8,
 9.21888888888889,
 9.133,
 7.786,
 9.167,
 9.937999999999999,
 9.367,
 6.978,
 9.253,
 8.747,
 9.66,
 8.32,
 10.0,
 10.0,
 10.0,
 9.261,
 9.683,
 6.866,
 9.683,
 9.875,
 10.0,
 9.517,
 9.675,
 8.608,
 9.933,
 10.0,
 9.315000000000001,
 7.014,
 8.049000000000001,
 9.815,
 8.492,
 10.0,
 9.239,
 9.142,
 9.283,
 8.617,
 9.208,
 10.0,
 8.975,
 9.57,
 9.408333333333333,
 8.739,
 7.642,
 10.0,
 5.672,
 8.206,
 9.047,
 7.912000000000001,
 9.322,
 8.795,
 5.556,
 8.596666666666666,
 9.275,
 8.142,
 8.653333333333334,
 9.3,
 9.2,
 9.258,
 9.222222222222221,
 9.065999999999999,
 9.136,
 9.461,
 9.639,
 9.8,
 5.668,
 7.675,
 9.302000000000001,
 9.275,
 9.59375,
 9.097,
 9.808,
 9.564,
 9.038,
 8.538571428571428,
 9.875,
 9.733,
 8.297,
 9.139,
 8.85,
 10.0,
 7.390000000000001,
 9.055,
 8.59,
 6.843999999999999,
 5.456,
 9.145,
 8.461,
 9.733,
 9.4,
 9.4,
 8.741111111111111,
 8.097,
 10.0,
 9.275,
 9.386,
 7.911,
 8.742,
 8.556000000000001,
 9.206,
 9.158000000000001

In [10]:
#Cluster Start 3 (LARGE 5-6)
tg3=df3_grade_total.groupby('Cluster (START)').get_group(3)

tg3
col=tg3.columns
large_grade=tg3[col[12:13:1]]
large_grade.dropna(inplace=True)

meanlarge_grade = large_grade.mean(axis=0)#μεσος ορος τελικής βαθμολογίας μαθητων large στις 5 ασκήσεις + 5 Quizzes

large_grade_list=[]
for i, row in large_grade.iterrows():
    large_grade_list.append(large_grade.at[i,'mean'])
large_grade_list

<ipython-input-10-2b1dd97905fe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  large_grade.dropna(inplace=True)


[8.864,
 8.175,
 9.7,
 9.108,
 6.767,
 8.131,
 8.767,
 9.6,
 8.275,
 10.0,
 9.553,
 9.697,
 9.078,
 7.333,
 10.0,
 8.033,
 7.325999999999999,
 8.9175,
 9.375,
 9.733,
 9.55,
 9.291,
 9.3,
 10.0,
 7.4079999999999995,
 8.25,
 9.75,
 9.475,
 9.764,
 9.6,
 6.256,
 8.571428571428571,
 7.583,
 8.417,
 8.839,
 8.606,
 8.5,
 5.953,
 8.253,
 9.8,
 7.5,
 9.875,
 9.145555555555555,
 9.4,
 9.089,
 5.681,
 9.167,
 8.131,
 7.078,
 10.0,
 9.119,
 9.145,
 8.995000000000001,
 6.8,
 10.0,
 8.942,
 7.615714285714286,
 7.339,
 7.752,
 9.7,
 9.597,
 9.2,
 9.572,
 9.3,
 10.0,
 9.055,
 9.106,
 9.8,
 9.411,
 8.083333333333334,
 10.0,
 9.275,
 9.5,
 9.63,
 5.867,
 10.0,
 8.7,
 9.889,
 7.014,
 7.78875,
 9.475,
 9.9,
 10.0,
 9.55,
 9.239,
 10.0,
 9.097,
 9.453,
 9.45,
 9.608,
 6.0,
 9.8,
 8.8,
 8.8,
 6.25,
 8.281,
 8.633,
 8.8,
 8.8,
 8.808,
 8.837142857142856,
 7.7219999999999995,
 9.933,
 7.756,
 7.411,
 9.0,
 9.305555555555555,
 8.984285714285715,
 9.9,
 9.5,
 8.6,
 9.867,
 8.85,
 7.756,
 9.875,
 9.575,
 8.83

In [12]:
#Cluster Start 4 (XLARGE <6)
tg4=df3_grade_total.groupby('Cluster (START)').get_group(4)

tg4
col=tg4.columns
xlarge_grade=tg4[col[12:13:1]]
xlarge_grade.dropna(inplace=True)

meanxlarge_grade = xlarge_grade.mean(axis=0)#μεσος ορος τελικής βαθμολογίας μαθητων xlarge στις 5 ασκήσεις + 5 Quizzes

xlarge_grade_list=[]
for i, row in xlarge_grade.iterrows():
    xlarge_grade_list.append(xlarge_grade.at[i,'mean'])
xlarge_grade_list

<ipython-input-12-3d07cb66f13f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xlarge_grade.dropna(inplace=True)


[7.164,
 9.367,
 9.683,
 8.806000000000001,
 9.238,
 6.328,
 6.892,
 10.0,
 10.0,
 9.8,
 9.108,
 9.8325,
 6.473333333333333,
 10.0,
 8.835,
 10.0,
 9.3,
 7.3469999999999995,
 9.1,
 9.9,
 9.6,
 9.0,
 10.0,
 6.889,
 9.775,
 9.775,
 9.666666666666666,
 10.0,
 10.0,
 10.0,
 9.522,
 9.933,
 9.9,
 8.625,
 9.575,
 9.9,
 10.0,
 7.672,
 10.0,
 9.875,
 8.0,
 9.406,
 6.553,
 9.733]

# one way ANOVA test

In [14]:
# Shapiro-Wilk normality test
stats.shapiro(low_grade_list), stats.shapiro(medium_grade_list), stats.shapiro(large_grade_list), stats.shapiro(xlarge_grade_list)

#Όλες οι τιμές p είναι μικρότερες από το όριο a = 0,05, 
#επομένως δεν «αποτυγχάνουμε να απορρίψουμε» τη μηδενική υπόθεση 
#τα δείγματα προέρχονται από πληθυσμούς που δεν ακολουθούν την κανονική κατανομή

(ShapiroResult(statistic=0.9536237716674805, pvalue=0.055829934775829315),
 ShapiroResult(statistic=0.8651453852653503, pvalue=9.193015667034657e-18),
 ShapiroResult(statistic=0.8762716054916382, pvalue=6.189828919681258e-17),
 ShapiroResult(statistic=0.7592152953147888, pvalue=4.220349012484803e-07))

In [15]:
# Levene variance test 
stats.levene(low_grade_list, medium_grade_list, large_grade_list, xlarge_grade_list)

#Η τιμή p είναι μεγαλύτερη από το όριο a = 0,05, 
#επομένως «αποτυγχάνουμε να απορρίψουμε» τη μηδενική υπόθεση 
#τα δείγματα προέρχονται από πληθυσμούς με την ίδια διακύμανση

LeveneResult(statistic=0.9270338892114355, pvalue=0.4270993720377011)

In [16]:
#Apply one way ANOVA by calling the f_oneway() method
F, p = stats.f_oneway(low_grade_list, medium_grade_list, large_grade_list, xlarge_grade_list)
print('F statistic = {:5.3f} and probability p = {:5.3f}'.format(F, p)) 

F statistic = 4.820 and probability p = 0.002


# Kruskal Wallis test

In [17]:
stats.kruskal(low_grade_list, medium_grade_list, large_grade_list, xlarge_grade_list)

KruskalResult(statistic=18.901585705430843, pvalue=0.00028651208579361104)

# Dunn’s test

In [18]:
#perform Dunn's test using a Bonferonni correction for the p-values

data = [low_grade_list, medium_grade_list, large_grade_list, xlarge_grade_list]
data

sp.posthoc_dunn(data , p_adjust = 'bonferroni')

,1,2,3,4
1,1.000000,0.002654,0.007226,0.000146
2,0.002654,1.000000,1.000000,0.186602
3,0.007226,1.000000,1.000000,0.093493
4,0.000146,0.186602,0.093493,1.000000


# Cohen's Effect Size

In [25]:
from numpy import mean
from numpy import var
from math import sqrt

n1,n2= len(low_grade_list), len(medium_grade_list)
s1,s2= var(low_grade_list, ddof=1), var(medium_grade_list, ddof=1)
s= sqrt(((n1-1)*s1+(n2-1)*s2)/(n1+n2-2))
u1,u2= mean(low_grade_list), mean(medium_grade_list)
d= (u1-u2)/s

cohen= abs(d)
if cohen<0.10:
    print('Very Small Effect Size=',cohen)
elif cohen<0.1:
    print('Small Effect Size=',cohen)
elif cohen<0.50:
    print('Medium Effect Size=',cohen)
elif cohen<0.80:
    print('Large Effect Size=',cohen)
else:
    print('Very Large=', cohen)

Large Effect Size= 0.5296696860069191


In [26]:
n1,n2= len(low_grade_list), len(large_grade_list)
s1,s2= var(low_grade_list, ddof=1), var(large_grade_list, ddof=1)
s= sqrt(((n1-1)*s1+(n2-1)*s2)/(n1+n2-2))
u1,u2= mean(low_grade_list), mean(large_grade_list)
d= (u1-u2)/s

cohen= abs(d)
if cohen<0.10:
    print('Very Small Effect Size=',cohen)
elif cohen<0.1:
    print('Small Effect Size=',cohen)
elif cohen<0.50:
    print('Medium Effect Size=',cohen)
elif cohen<0.80:
    print('Large Effect Size=',cohen)
else:
    print('Very Large=', cohen)

Large Effect Size= 0.5225077199015116


In [27]:
n1,n2= len(low_grade_list), len(xlarge_grade_list)
s1,s2= var(low_grade_list, ddof=1), var(xlarge_grade_list, ddof=1)
s= sqrt(((n1-1)*s1+(n2-1)*s2)/(n1+n2-2))
u1,u2= mean(low_grade_list), mean(xlarge_grade_list)
d= (u1-u2)/s

cohen= abs(d)
if cohen<0.10:
    print('Very Small Effect Size=',cohen)
elif cohen<0.1:
    print('Small Effect Size=',cohen)
elif cohen<0.50:
    print('Medium Effect Size=',cohen)
elif cohen<0.80:
    print('Large Effect Size=',cohen)
else:
    print('Very Large=', cohen)

Large Effect Size= 0.6344199216584834


In [34]:
n1,n2= len(medium_grade_list), len(large_grade_list)
s1,s2= var(medium_grade_list, ddof=1), var(large_grade_list, ddof=1)
s= sqrt(((n1-1)*s1+(n2-1)*s2)/(n1+n2-2))
u1,u2= mean(medium_grade_list), mean(large_grade_list)
d= (u1-u2)/s

cohen= abs(d)
if cohen<0.10:
    print('Very Small Effect Size=',cohen)
elif cohen<0.1:
    print('Small Effect Size=',cohen)
elif cohen<0.50:
    print('Medium Effect Size=',cohen)
elif cohen<0.80:
    print('Large Effect Size=',cohen)
else:
    print('Very Large=', cohen)

Very Small Effect Size= 0.01922203770632086


In [29]:
n1,n2= len(medium_grade_list), len(xlarge_grade_list)
s1,s2= var(medium_grade_list, ddof=1), var(xlarge_grade_list, ddof=1)
s= sqrt(((n1-1)*s1+(n2-1)*s2)/(n1+n2-2))
u1,u2= mean(medium_grade_list), mean(xlarge_grade_list)
d= (u1-u2)/s

cohen= abs(d)
if cohen<0.10:
    print('Very Small Effect Size=',cohen)
elif cohen<0.1:
    print('Small Effect Size=',cohen)
elif cohen<0.50:
    print('Medium Effect Size=',cohen)
elif cohen<0.80:
    print('Large Effect Size=',cohen)
else:
    print('Very Large=', cohen)

Medium Effect Size= 0.17120038444983357


In [30]:
n1,n2= len(large_grade_list), len(xlarge_grade_list)
s1,s2= var(large_grade_list, ddof=1), var(xlarge_grade_list, ddof=1)
s= sqrt(((n1-1)*s1+(n2-1)*s2)/(n1+n2-2))
u1,u2= mean(large_grade_list), mean(xlarge_grade_list)
d= (u1-u2)/s

cohen= abs(d)
if cohen<0.10:
    print('Very Small Effect Size=',cohen)
elif cohen<0.1:
    print('Small Effect Size=',cohen)
elif cohen<0.50:
    print('Medium Effect Size=',cohen)
elif cohen<0.80:
    print('Large Effect Size=',cohen)
else:
    print('Very Large=', cohen)

Medium Effect Size= 0.19425188261440607


# In the first directions of the pairs the results of effect size are <0 , in the opposite directions of the pairs the results of effect size are >0 :   
low-medium --> Very Small: -0.5296696860069191            medium-low --> Large: 0.5296696860069191
low-large --> Very Small:  -0.5225077199015116             large-low --> Large: 0.5225077199015116
low-xlarge --> Very Small: -0.6344199216584834             xlarge-low --> Large: 0.6344199216584834
medium-large --> Very Small: 0.01922203770632086           large-medium --> Very Small: -0.01922203770632086
medium-xlarge --> Very Small: -0.17120038444983357        xlarge-medium --> Medium: 0.17120038444983357
large-xlarge --> Very Small: -0.19425188261440607           xlarge-large --> Medium: 0.19425188261440607